<a href="https://colab.research.google.com/github/anncam/Transformers/blob/main/transformers_retaurant_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Packages Required

In [29]:
!pip3 install ktrain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
import numpy as np
import pandas as pd
import missingno as msno
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import plotly.graph_objects as go
#import plotly.express as px
import matplotlib.pyplot as plt
import spacy
import tensorflow as tf
from wordcloud import WordCloud, STOPWORDS 
import ktrain
from ktrain import text

from collections import Counter
%matplotlib inline

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
#df = pd.read_csv('../input/trip-advisor-hotel-reviews/tripadvisor_hotel_reviews.csv')
df = pd.read_csv('drive/My Drive/Colab Notebooks/data/New_York_reviews.csv')
df.head()

,Unnamed: 0,parse_count,restaurant_name,rating_review,sample,review_id,title_review,review_preview,review_full,date,city,url_restaurant,author_id
0,1,2,Lido,5,Positive,review_773559838,A Regular Treat,My wife and I have been eating dinner frequent...,My wife and I have been eating dinner frequent...,"October 8, 2020",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_0
1,2,3,Lido,4,Positive,review_769429529,Good neighborhood spot!,Came with family for Labor Day weekend brunch ...,Came with family for Labor Day weekend brunch ...,"September 8, 2020",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_1
2,3,4,Lido,1,Negative,review_745700258,Disappointing,Food was mediocre at best. The lamb chops are...,Food was mediocre at best. The lamb chops are ...,"February 17, 2020",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_2
3,4,5,Lido,5,Positive,review_728859349,What a find in Harlem,My co-workers were volunteering at a foodbank ...,My co-workers were volunteering at a foodbank ...,"November 25, 2019",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_3
4,5,6,Lido,5,Positive,review_728429643,Lunch,Lido is an intimate boutique style restaurant....,Lido is an intimate boutique style restaurant....,"November 23, 2019",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_4


In [33]:
df.shape

(510463, 13)

In [8]:
df.isnull().sum().any()

True

## Taking a subset of data since its so large

In [34]:
df = df.sample(n=10000)
df.shape

(10000, 13)

In [35]:
df.head()

,Unnamed: 0,parse_count,restaurant_name,rating_review,sample,review_id,title_review,review_preview,review_full,date,city,url_restaurant,author_id
43918,44601,44602,Sean_s_Bar_And_Kitchen,5,Positive,review_582606085,Great atmosphere,As my husband and I were walking around search...,As my husband and I were walking around search...,"May 25, 2018",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_36995
274649,278447,278448,Casa_Nonna,4,Positive,review_513850592,Lovely Restaurant Week Lunch,Had a nice time as a solo diner during Restaur...,Had a nice time as a solo diner during Restaur...,"August 16, 2017",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_24092
205536,208426,208427,Don_Antonio,5,Positive,review_644143910,Amazing pizzas,This is the best pizza I have tasted in New Yo...,This is the best pizza I have tasted in New Yo...,"January 6, 2019",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_132514
117557,119161,119162,44_X,3,Negative,review_609936469,Good but bit pricey.,Good food service good. Pricey for what it was...,Good food service good. Pricey for what it was...,"August 24, 2018",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_46319
477248,483948,483949,Neapolitan_Express,5,Positive,review_320721997,Amazing pizza!,The crust is just perfect. You can pick your t...,The crust is just perfect. You can pick your t...,"October 21, 2015",New_York_City_New_York,https://www.tripadvisor.com/Restaurant_Review-...,UID_20109


## Removing unnecessary rows- i only need the full review and the rating

In [38]:
df=df[["review_full", "rating_review" ]]
df.head()

,review_full,rating_review
43918,As my husband and I were walking around search...,5
274649,Had a nice time as a solo diner during Restaur...,4
205536,This is the best pizza I have tasted in New Yo...,5
117557,Good food service good. Pricey for what it was...,3
477248,The crust is just perfect. You can pick your t...,5
497596,Of course after asking my hotelI got this nice...,4
111531,"Walking from Central Park, past looking for It...",5
154192,Ate here several times during my stay in NYC a...,4
386641,This place is rated to have one of the best Eg...,5
205177,We needed food - fast - and picked this place....,5


# Sentiment Visualisation

In [39]:
pos = [4, 5]
neg = [1, 2]
neu = [3]

def sentiment(rating):
  if rating in pos:
    return 2
  elif rating in neg:
    return 0
  else:
    return 1  

In [40]:
df['Sentiment'] = df['rating_review'].apply(sentiment)
df.head()

,review_full,rating_review,Sentiment
43918,As my husband and I were walking around search...,5,2
274649,Had a nice time as a solo diner during Restaur...,4,1
205536,This is the best pizza I have tasted in New Yo...,5,2
117557,Good food service good. Pricey for what it was...,3,1
477248,The crust is just perfect. You can pick your t...,5,2


In [41]:
fig = go.Figure([go.Bar(x=df.Sentiment.value_counts().index, y=df.Sentiment.value_counts().tolist())])
fig.update_layout(
    title="Values in each Sentiment",
    xaxis_title="Sentiment",
    yaxis_title="Values")
fig.show()


2 - Positive (4, 5)<br>
1 - Neutral (3)<br>
0 - Negative (1, 2)

More good reviews that bad


## Wordclouds of each Sentiment

Let us now look at the word distribution overrall and for each sentiment.

In [42]:
import spacy
nlp = spacy.load('en_core_web_sm')

def normalize(msg):
    
    doc = nlp(msg)
    res = []
    
    for token in doc:
        if(token.is_stop or token.is_punct or token.is_space):
            pass
        else:
            res.append(token.lemma_.lower())
            
    return res

In [43]:
df['review_full'] = df['review_full'].apply(normalize)
df.head()

,review_full,rating_review,Sentiment
43918,"[husband, walk, search, place, naturally, find...",5,2
274649,"[nice, time, solo, diner, restaurant, week, pl...",4,1
205536,"[good, pizza, taste, new, york, place, offer, ...",5,2
117557,"[good, food, service, good, pricey, nice, ambi...",3,1
477248,"[crust, perfect, pick, topping, personalized, ...",5,2


In [44]:
words_collection = Counter([item for sublist in df['review_full'] for item in sublist])
freq_word_df = pd.DataFrame(words_collection.most_common(15))
freq_word_df.columns = ['frequently_used_word','count']

freq_word_df.style.background_gradient(cmap='PuBuGn', low=0, high=0, axis=0, subset=None)

,frequently_used_word,count
0,good,7073
1,food,6916
2,great,4697
3,restaurant,4319
4,service,4220
5,place,4169
6,time,2754
7,order,2499
8,table,2456
9,eat,2213


In [47]:
word_list = [item for sublist in df['review_full'] for item in sublist]
word_string = " ".join(word_list)

wordcloud = WordCloud(stopwords=STOPWORDS,
                          background_color='white', 
                      max_words=60000, 
                      width=1000,
                      height=650
                         ).generate(word_string)

## Positive Sentiment Wordcloud

In [48]:
pos_df = df[df['Sentiment'] == 2]
words_collection = Counter([item for sublist in pos_df['review_full'] for item in sublist])
freq_word_df = pd.DataFrame(words_collection.most_common(15))
freq_word_df.columns = ['frequently_used_word','count']

freq_word_df.style.background_gradient(cmap='PuBuGn', low=0, high=0, axis=0, subset=None)

,frequently_used_word,count
0,good,5023
1,food,4690
2,great,3686
3,service,2916
4,restaurant,2913
5,place,2891
6,time,1895
7,delicious,1583
8,eat,1489
9,table,1458


In [49]:
word_list_pos = [item for sublist in pos_df['review_full'] for item in sublist]
word_string_pos = " ".join(word_list)

wordcloud = WordCloud(stopwords=STOPWORDS,
                          background_color='white', 
                      max_words=40000, 
                      width=1000,
                      height=650
                         ).generate(word_string_pos)

## Negative Sentiment Wordcloud

In [52]:
neg_df = df[df['Sentiment'] == 0]
words_collection = Counter([item for sublist in neg_df['review_full'] for item in sublist])
freq_word_df = pd.DataFrame(words_collection.most_common(15))
freq_word_df.columns = ['frequently_used_word','count']

freq_word_df.style.background_gradient(cmap='PuBuGn', low=0, high=0, axis=0, subset=None)

,frequently_used_word,count
0,food,566
1,restaurant,444
2,table,356
3,place,355
4,order,353
5,service,311
6,good,299
7,time,269
8,ask,238
9,come,234


In [54]:
word_list_neg = [item for sublist in neg_df['review_full'] for item in sublist]
word_string_neg = " ".join(word_list)

wordcloud = WordCloud(stopwords=STOPWORDS,
                          background_color='white', 
                      max_words=10000, 
                      width=1000,
                      height=650
                         ).generate(word_string_neg)

Some obvious words like "bad", "problem" can be seen in this wordcloud. Maybe when we expand the vocabulary we can clean the text a little better.

## BERT model 
- from the ktrain module
- must turn reviews into strings

In [56]:
df['review_full'] = df['review_full'].apply(lambda m: " ".join(m))
df.head()

,review_full,rating_review,Sentiment
43918,husband walk search place naturally find appea...,5,2
274649,nice time solo diner restaurant week pleasant ...,4,1
205536,good pizza taste new york place offer average ...,5,2
117557,good food service good pricey nice ambience cl...,3,1
477248,crust perfect pick topping personalized pizza ...,5,2


# Training BERT model

In [57]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(df, 
                                                                    'review_full',
                                                                    label_columns=['Sentiment'],
                                                                    preprocess_mode='bert')

['Sentiment_0', 'Sentiment_1', 'Sentiment_2']
        Sentiment_0  Sentiment_1  Sentiment_2
306818          0.0          1.0          0.0
263145          0.0          1.0          0.0
464703          0.0          1.0          0.0
247921          0.0          0.0          1.0
260931          0.0          0.0          1.0
['Sentiment_0', 'Sentiment_1', 'Sentiment_2']
        Sentiment_0  Sentiment_1  Sentiment_2
174150          0.0          0.0          1.0
152181          0.0          0.0          1.0
396772          0.0          0.0          1.0
26934           0.0          0.0          1.0
50490           0.0          0.0          1.0
preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [58]:
model = text.text_classifier(name='bert',
                             train_data=(x_train, y_train),
                             preproc=preproc)

Is Multi-Label? False
maxlen is 400
done.


In [59]:
learner = ktrain.get_learner(model=model,
                             train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)

In [60]:
learner.fit_onecycle(lr=2e-5,
                     epochs=1)



begin training using onecycle policy with max lr of 2e-05...
1500/1500 [==============================] - 941s 616ms/step - loss: 0.6167 - accuracy: 0.7473 - val_loss: 0.5556 - val_accuracy: 0.7680


# Results

Test accuracy of with 71.22% in Training and 73% in Validation when only using a sample of 1000 reviews. (took 3 minutes)

Tried again with 10,000 reviews. Got for the 74.73% accuracy for the training set and 76.8% accuracy for the validation set, so it improved slightly with a larger sample of data. (took 15 minutes) 